In [3]:
!pip install keras-tuner

     |████████████████████████████████| 97 kB 3.4 MB/s 


**Create CNN Model and Optimize it using Keras Tuner**

In [4]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  after removing the cwd from sys.path.


In [5]:
fashion_mnist=keras.datasets.fashion_mnist

In [6]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [9]:
fashion_mnist.load_data()

((array([[[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         ...,
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0

In [10]:
train_images= train_images/255.0
test_images= test_images/255.0

In [11]:
train_images[0].shape

(28, 28)

In [12]:
train_images= train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [13]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [14]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [15]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [16]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 23s]
val_accuracy: 0.859499990940094

Best val_accuracy So Far: 0.9075000286102295
Total elapsed time: 00h 02m 00s
INFO:tensorflow:Oracle triggered exit


In [24]:
model=tuner_search.get_best_models(num_models=1)[0]

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 64)        1664      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 30976)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                1486896   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                490       
Total params: 1,525,978
Trainable params: 1,525,978
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1695 - accuracy: 0.9366 - val_loss: 0.2567 - val_accuracy: 0.9148
Epoch 5/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1350 - accuracy: 0.9486 - val_loss: 0.2864 - val_accuracy: 0.9088
Epoch 6/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.1035 - accuracy: 0.9612 - val_loss: 0.3265 - val_accuracy: 0.9105
Epoch 7/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0769 - accuracy: 0.9699 - val_loss: 0.3422 - val_accuracy: 0.9160
Epoch 8/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0641 - accuracy: 0.9762 - val_loss: 0.3696 - val_accuracy: 0.9095
Epoch 9/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0463 - accuracy: 0.9836 - val_loss: 0.4096 - val_accuracy: 0.9087
Epoch 10/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0405 - accuracy: 0.9851 - val_loss: 0.4922 - val_accuracy